# 範例 : 計程車費率預測
https://www.kaggle.com/c/new-york-city-taxi-fare-prediction

# [作業目標]
- 使用並觀察特徵組合, 在計程車費率預測競賽的影響

# [作業重點]
- 仿造範例並參考今日課程內容, 使用經緯度一圈的長度比的概念造出新特徵, 觀察有什麼影響 (In[6], Out[6])
- 只使用上面所造的這個新特徵, 觀察有什麼影響 (In[7], Out[7])

In [1]:
# 做完特徵工程前的所有準備
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

data_path = ''
df = pd.read_csv(data_path + 'taxi_data1.csv')

train_Y = df['fare_amount']
df = df.drop(['fare_amount'] , axis=1)
df.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2011-10-21 23:54:10 UTC,-73.990580,40.761071,-73.981128,40.758634,2
1,2015-02-03 10:42:03 UTC,-73.988403,40.723431,-73.989647,40.741695,1
2,2014-03-16 18:58:58 UTC,-74.015785,40.715110,-74.012029,40.707888,2
3,2009-06-13 16:10:54 UTC,-73.977322,40.787275,-73.958030,40.778838,3
4,2014-06-12 03:25:56 UTC,-73.989683,40.729717,-73.982490,40.761887,3


In [2]:
# 時間特徵分解方式:使用datetime
df['pickup_datetime'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S UTC'))
df['pickup_year'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%Y')).astype('int64')
df['pickup_month'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
df['pickup_day'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%d')).astype('int64')
df['pickup_hour'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%H')).astype('int64')
df['pickup_minute'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%M')).astype('int64')
df['pickup_second'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%S')).astype('int64')
df.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second
0,2011-10-21 23:54:10,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10
1,2015-02-03 10:42:03,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3
2,2014-03-16 18:58:58,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58
3,2009-06-13 16:10:54,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54
4,2014-06-12 03:25:56,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56


In [3]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果
df = df.drop(['pickup_datetime'] , axis=1)
scaler = MinMaxScaler()
train_X = scaler.fit_transform(df)
Linear = LinearRegression()
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
GDBT = GradientBoostingRegressor()
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.026876871475641616
Gradient Boosting Reg Score : 0.7120673593060853


In [4]:
# 增加緯度差, 經度差, 座標距離等三個特徵
df['longitude_diff'] = df['dropoff_longitude'] - df['pickup_longitude']
df['latitude_diff'] = df['dropoff_latitude'] - df['pickup_latitude']
df['distance_2D'] = (df['longitude_diff']**2 + df['latitude_diff']**2)**0.5
df[['distance_2D', 'longitude_diff', 'latitude_diff', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']].head()

,distance_2D,longitude_diff,latitude_diff,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,0.009761,0.009452,-0.002437,-73.990580,40.761071,-73.981128,40.758634
1,0.018307,-0.001244,0.018265,-73.988403,40.723431,-73.989647,40.741695
2,0.008140,0.003756,-0.007222,-74.015785,40.715110,-74.012029,40.707888
3,0.021056,0.019292,-0.008437,-73.977322,40.787275,-73.958030,40.778838
4,0.032964,0.007193,0.032170,-73.989683,40.729717,-73.982490,40.761887


In [5]:
# 結果 : 準確度上升
train_X = scaler.fit_transform(df)
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.027479693774541868
Gradient Boosting Reg Score : 0.805120218061403


# 作業1
* 參考今日教材，試著使用經緯度一圈的長度比這一概念，組合出一個新特徵，再觀察原特徵加上新特徵是否提升了正確率?
linear increase but gradient decrease why?

In [6]:
import math
"""
Your Code Here, set new character at df['distance_real']
"""
df["distance_real"]=(df['longitude_diff']**2 + (0.75756*df['latitude_diff'])**2)**0.5

# 觀察結果 
train_X = scaler.fit_transform(df)
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')
df.head()

Linear Reg Score : 0.03025316079375029
Gradient Boosting Reg Score : 0.8023621112896449


,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second,longitude_diff,latitude_diff,distance_2D,distance_real
0,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10,0.009452,-0.002437,0.009761,0.009631
1,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3,-0.001244,0.018265,0.018307,0.013892
2,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58,0.003756,-0.007222,0.008140,0.006636
3,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54,0.019292,-0.008437,0.021056,0.020323
4,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56,0.007193,0.032170,0.032964,0.025410


# 作業2
* 試著只使用新特徵估計目標值(忽略原特徵)，效果跟作業1的結果比較起來效果如何?
linear 大幅降低，gradient 小幅降低?

In [7]:
train_X = scaler.fit_transform(df[['distance_real']])
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.001401340887114521
Gradient Boosting Reg Score : 0.7360082494416365


In [30]:
#將地理座標系轉換成直角座標系

In [8]:
#df=df.drop(["distance_real"],axis=1)
#print(math.cos(math.radians(40.75)))
pxx=[]
pyy=[]
pzz=[]
dxx=[]
dyy=[]
dzz=[]
for i,j,k,l in zip(df["pickup_latitude"],df["pickup_longitude"],df["dropoff_latitude"],df["dropoff_longitude"]):
    px=math.cos(math.radians(i))*math.cos(math.radians(j))
    py=math.cos(math.radians(i))*math.sin(math.radians(j))
    pz=math.sin(math.radians(i))
    pxx.append(px)
    pyy.append(py)
    pzz.append(pz)

    dx=math.cos(math.radians(k))*math.cos(math.radians(l))
    dy=math.cos(math.radians(k))*math.sin(math.radians(l))
    dz=math.sin(math.radians(k))
    dxx.append(dx)
    dyy.append(dy)
    dzz.append(dz)
    
df["pickup_x"]=pxx
df["pickup_y"]=pyy
df["pickup_z"]=pzz

df["dropoff_x"]=dxx
df["dropoff_y"]=dyy
df["dropoff_z"]=dzz

df["3D_xdiff"]=df["dropoff_x"]-df["pickup_x"]
df["3D_ydiff"]=df["dropoff_y"]-df["pickup_y"]
df["3D_zdiff"]=df["dropoff_z"]-df["pickup_z"]

df["distance_3D"]=(df["3D_xdiff"]**2 + df["3D_ydiff"]**2 + df["3D_zdiff"]**2)**0.5
df.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,...,pickup_x,pickup_y,pickup_z,dropoff_x,dropoff_y,dropoff_z,3D_xdiff,3D_ydiff,3D_zdiff,distance_3D
0,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,...,0.208898,-0.728063,0.652906,0.209026,-0.728055,0.652874,0.000128,0.000008,-0.000032,0.000132
1,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,...,0.209044,-0.728467,0.652408,0.208971,-0.728271,0.652650,-0.000073,0.000195,0.000242,0.000319
2,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,...,0.208722,-0.728658,0.652298,0.208792,-0.728723,0.652203,0.000070,-0.000065,-0.000096,0.000135
3,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,...,0.208984,-0.727727,0.653252,0.209256,-0.727749,0.653141,0.000272,-0.000022,-0.000111,0.000294
4,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,...,0.209008,-0.728403,0.652492,0.208998,-0.728024,0.652917,-0.000010,0.000378,0.000425,0.000569


In [31]:
#try to compare only three different distance result

In [9]:
train_X = scaler.fit_transform(df[['distance_2D']])
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.0014280087770600636
Gradient Boosting Reg Score : 0.7354234771982402


In [10]:
train_X = scaler.fit_transform(df[['distance_real']])
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.001401340887114521
Gradient Boosting Reg Score : 0.7360082494416365


In [11]:
train_X = scaler.fit_transform(df[['distance_3D']])
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.0013472973637295381
Gradient Boosting Reg Score : 0.7149143843924612
